In [1]:
import cv2
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_csv('train.csv')
df.isnull().sum()

image_names         0
emergency_or_not    0
dtype: int64

In [3]:
y = df.iloc[:,0].values
y.shape
images=[]
labels = df.iloc[:,1].values

In [4]:
path = 'images'
# f = os.path.join(path,y[0])


In [5]:
for i in y:
    if os.path.exists(os.path.join(path,i)):
        image = cv2.imread(os.path.join(path,i))
        img = cv2.resize(image,(64,64))
        img = img/256
        images.append(img)
        

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2,random_state=42)

In [7]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [8]:
X_train[0].shape

(64, 64, 3)

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import tensorflow as tf
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD

Using TensorFlow backend.


In [10]:
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 32)          0         
________________________________________________

In [11]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
                featurewise_center=False,  # set input mean to 0 over the dataset
                samplewise_center=False,  # set each sample mean to 0
                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                samplewise_std_normalization=False,  # divide each input by its std
                zca_whitening=False,  # apply ZCA whitening
                rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
                width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
                height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
                horizontal_flip=True,# randomly flip images
                vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

In [12]:
len(X_train)

1316

In [13]:
classifier.fit_generator(datagen.flow(X_train, y_train,
                    batch_size=32),
                    samples_per_epoch=2000,
                    nb_epoch=100,
                    validation_data=(X_test, y_test))

Instructions for updating:
Use tf.cast instead.


C:\Users\PAVAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
C:\Users\PAVAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=(array([[[..., steps_per_epoch=62, epochs=100)`
  """


Epoch 1/100
62/62 [==============================] - 9s 139ms/step - loss: 0.6496 - accuracy: 0.6276 - val_loss: 0.6088 - val_accuracy: 0.6515
Epoch 2/100
62/62 [==============================] - 8s 124ms/step - loss: 0.6020 - accuracy: 0.6800 - val_loss: 0.5675 - val_accuracy: 0.7273
Epoch 3/100
62/62 [==============================] - 8s 128ms/step - loss: 0.5556 - accuracy: 0.7117 - val_loss: 0.5803 - val_accuracy: 0.6667
Epoch 4/100
62/62 [==============================] - 8s 127ms/step - loss: 0.5624 - accuracy: 0.7059 - val_loss: 0.5342 - val_accuracy: 0.7424
Epoch 5/100
62/62 [==============================] - 8s 132ms/step - loss: 0.5320 - accuracy: 0.7439 - val_loss: 0.5371 - val_accuracy: 0.7364
Epoch 6/100
62/62 [==============================] - 8s 130ms/step - loss: 0.5256 - accuracy: 0.7562 - val_loss: 0.4933 - val_accuracy: 0.7576
Epoch 7/100
62/62 [==============================] - 8s 130ms/step - loss: 0.5185 - accuracy: 0.7386 - val_loss: 0.4771 - val_accuracy: 0.7909

Epoch 58/100
62/62 [==============================] - 9s 138ms/step - loss: 0.2269 - accuracy: 0.9103 - val_loss: 0.3158 - val_accuracy: 0.8515
Epoch 59/100
62/62 [==============================] - 8s 134ms/step - loss: 0.2022 - accuracy: 0.9227 - val_loss: 0.3587 - val_accuracy: 0.8727
Epoch 60/100
62/62 [==============================] - 8s 136ms/step - loss: 0.2078 - accuracy: 0.9136 - val_loss: 0.3785 - val_accuracy: 0.8303
Epoch 61/100
62/62 [==============================] - 8s 136ms/step - loss: 0.1820 - accuracy: 0.9248 - val_loss: 0.3448 - val_accuracy: 0.8636
Epoch 62/100
62/62 [==============================] - 8s 135ms/step - loss: 0.1910 - accuracy: 0.9170 - val_loss: 0.3629 - val_accuracy: 0.8606
Epoch 63/100
62/62 [==============================] - 8s 135ms/step - loss: 0.1778 - accuracy: 0.9376 - val_loss: 0.3090 - val_accuracy: 0.8697
Epoch 64/100
62/62 [==============================] - 8s 135ms/step - loss: 0.1773 - accuracy: 0.9253 - val_loss: 0.3679 - val_accuracy:

In [14]:
dataframe = pd.read_csv('test.csv')
dataframe.isnull().sum()

image_names    0
dtype: int64

In [15]:
dataframe.columns

Index(['image_names'], dtype='object')

In [16]:
test_set = dataframe.iloc[:,0].values
images_test=[]
test_set.shape

(706,)

In [17]:
test_set.size

706

In [18]:
for i in test_set:
    if os.path.exists(os.path.join(path,i)):
        image = cv2.imread(os.path.join(path,i))
        img = cv2.resize(image,(64,64))
        img = img/256
        images_test.append(img)
#     print(os.path.join(path,i))
# temp = os.path.join(path, test_set[0][0])


In [19]:
X_final = np.array(images_test)

In [20]:
ans = classifier.predict_classes(X_final)
ans = np.resize(ans, (706,))

In [21]:
# dataframe['image_names'].shape
ans.shape

(706,)

In [22]:
submit = pd.DataFrame({
    
    'image_names' : dataframe['image_names'],
    'emergency_or_not' : ans
})

In [23]:
dataframe['image_names'].shape

(706,)

In [26]:
submit.to_csv('submission100_adam.csv', index = False)

In [27]:
submit.describe()

,emergency_or_not
count,706.000000
mean,0.364023
std,0.481496
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000
